### Packages Required

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras.models import Model
from keras.layers import Input,LSTM,Dense

In [2]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = ('C:/Users/Nithin/Downloads/Machine Transalation_AI_Seq2Seq/Datasets/ara-eng/ara.txt')

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [4]:
with open(data_path,"r",encoding = "utf-8") as f:
    lines = f.read().split("\n")

In [5]:
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text,target_text, _ = line.split("\t")
    target_text = "\t"+ target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
print('no of samples : ',len(input_texts))
print('no of unique input words :',num_encoder_tokens)
print('no of unique Output words :',num_decoder_tokens)
print('max seq length for inputs :',max_encoder_seq_length)
print('max seq length for inputs :',max_decoder_seq_length)

no of samples :  10000
no of unique input words : 71
no of unique Output words : 94
max seq length for inputs : 37
max seq length for inputs : 54


In [17]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])

In [28]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype = 'float32'
)

In [29]:
decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype = 'float32'
)

In [30]:
decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype = 'float32'
)

In [31]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.0
        encoder_input_data[i,t + 1 :,input_token_index[" "]] = 1.0
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i,t-1,target_token_index[char]] = 1.0
            
    decoder_input_data[i,t + 1 :,target_token_index[" "]] = 1.0
    decoder_target_data[i,t:,target_token_index[" "]] = 1.0

In [32]:
len(decoder_target_data[decoder_target_data==1])

540000

In [33]:
len(decoder_input_data[decoder_input_data==1])

540000

In [25]:
encoder_input_data.shape

(10000, 37, 71)

In [26]:
decoder_input_data.shape

(10000, 54, 94)

In [27]:
decoder_target_data.shape

(10000, 54, 71)

In [34]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

M0 = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [35]:
M0.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics = ['accuracy'])

M0.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 37s 272ms/step - loss: 1.3752 - accuracy: 0.7007 - val_loss: 1.7668 - val_accuracy: 0.5792
Epoch 2/100
125/125 [==============================] - 33s 261ms/step - loss: 1.1221 - accuracy: 0.7133 - val_loss: 1.5823 - val_accuracy: 0.5853
Epoch 3/100
125/125 [==============================] - 31s 246ms/step - loss: 1.0470 - accuracy: 0.7295 - val_loss: 1.4842 - val_accuracy: 0.6040
Epoch 4/100
125/125 [==============================] - 31s 247ms/step - loss: 0.9759 - accuracy: 0.7435 - val_loss: 1.3984 - val_accuracy: 0.6318
Epoch 5/100
125/125 [==============================] - 31s 249ms/step - loss: 0.9250 - accuracy: 0.7550 - val_loss: 1.3666 - val_accuracy: 0.6395
Epoch 6/100
125/125 [==============================] - 35s 283ms/step - loss: 0.8924 - accuracy: 0.7626 - val_loss: 1.3016 - val_accuracy: 0.6569
Epoch 7/100
125/125 [==============================] - 41s 326ms/step - loss: 0.8697 - accuracy: 0.7675 - val_loss: 1.2824 -

In [36]:
M0.save('C:/Users/Nithin/Downloads/Machine Transalation_AI_Seq2Seq/Models/Machine Transalation.h5')

In [38]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [39]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [43]:
for seq_index in range(5):
    
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print(input_texts[seq_index])
    print(decoded_sentence)

1/1 [==============================] - 0s 60ms/step
-
Hi.
أنا أعرف من أين أن أشاره الحامة.

1/1 [==============================] - 0s 60ms/step
-
Run!
أنا أعرف من أين أن أشاره الحامة.

1/1 [==============================] - 0s 62ms/step
-
Duck!
أنا أعرف من أين أن أشاره الحاية.

1/1 [==============================] - 0s 53ms/step
-
Duck!
أنا أعرف من أين أن أشاره الحاية.

1/1 [==============================] - 0s 64ms/step
-
Duck!
أنا أعرف من أين أن أشاره الحاية.

